In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Air temperature multiple levels/*.nc'
air_temp=xr.open_mfdataset(path)


In [3]:
air_temp

<xarray.Dataset>
Dimensions:    (level: 17, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    air        (time, level, lat, lon) float32 dask.array<chunksize=(365, 17, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [4]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_air_temp = air_temp.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [5]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
air_temp_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'air_temp_1000hpa','air_temp_850hpa','air_temp_500hpa','air_temp_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_air_temp.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_air_temp.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    air_temp_1000hpa = chunk.sel(level=1000).air.values
    air_temp_850hpa = chunk.sel(level=850).air.values
    air_temp_500hpa = chunk.sel(level=500).air.values
    air_temp_300hpa = chunk.sel(level=300).air.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'air_temp_1000hpa': air_temp_1000hpa.ravel(),
        'air_temp_850hpa': air_temp_850hpa.ravel(),
        'air_temp_500hpa': air_temp_500hpa.ravel(),
        'air_temp_300hpa': air_temp_300hpa.ravel()
    })

    air_temp_df = air_temp_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
air_temp_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,air_temp_1000hpa,air_temp_850hpa,air_temp_500hpa,air_temp_300hpa
0,1981-01-01,27.5,85.0,292.924988,283.875000,260.900024,238.575012
1,1981-01-01,25.0,87.5,292.900024,284.075012,260.650024,238.775009
2,1981-01-01,22.5,90.0,291.100006,283.424988,260.225037,238.800003
3,1981-01-01,20.0,92.5,289.200012,282.600006,259.575012,238.475006
4,1981-01-01,27.5,95.0,289.224976,282.549988,258.799988,237.875000
...,...,...,...,...,...,...,...
306795,2022-12-31,20.0,85.0,297.174988,287.599976,268.224976,242.399994
306796,2022-12-31,27.5,87.5,295.500000,287.024994,268.349976,242.749985
306797,2022-12-31,25.0,90.0,293.949982,286.624969,268.449982,242.999985
306798,2022-12-31,22.5,92.5,293.674988,286.699982,268.399994,243.099976


In [9]:
air_temp_df[4000:4050]

,time,lat,lon,air_temp_1000hpa,air_temp_850hpa,air_temp_500hpa,air_temp_300hpa
4000,1981-07-20,27.5,85.0,305.400024,297.274994,272.375000,248.825012
4001,1981-07-20,25.0,87.5,305.825012,297.400024,272.275024,248.425003
4002,1981-07-20,22.5,90.0,305.325012,296.774994,272.075012,248.100006
4003,1981-07-20,20.0,92.5,304.875000,296.325012,271.600006,247.950012
4004,1981-07-20,27.5,95.0,305.049988,296.375000,270.950012,247.750000
4005,1981-07-20,25.0,85.0,300.750000,293.225006,271.450012,248.850006
4006,1981-07-20,22.5,87.5,301.200012,293.650024,271.125000,248.324997
4007,1981-07-20,20.0,90.0,301.125000,293.475006,271.025024,247.800003
4008,1981-07-20,27.5,92.5,301.100006,293.299988,270.849976,247.350006
4009,1981-07-20,25.0,95.0,301.700012,293.600006,270.450012,246.875000
